In [1]:
!pip install tf-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 kB 14.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.2 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.23.0-py3-none-any.whl size=697631 sha256=f6ffb984f31adce3dd7abd971a37a4026ef1a1aa9bd8255b5655653d2c1f8694
  Stored in directory: /root/.cache/pip/wheels/3d/6f/b4/3991d4fae11d0ecb0754c11cc1b4e7745012850da4efaaf0b1
Successfully built gym
  Attempting uninstall: tensorflow-probability
    Found existing installation: tensorflow-probability 0.22.0
    Uninstalling tensorflow-probability-0.22.0:
      Successfully uninstalled tensorflow-probability-0.22.0
  Attempting uninstall: pygame
    Found existing

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tf_agents.agents.dqn import dqn_agent
from tf_agents.environments import tf_py_environment
from tf_agents.environments import py_environment
from tf_agents.trajectories import time_step as ts
from tf_agents.environments import tf_environment
from tf_agents.environments import suite_gym
from tf_agents.environments import wrappers
from tf_agents.drivers import dynamic_episode_driver
from tf_agents.networks import q_network
from tf_agents.environments import utils
from scipy.sparse import issparse
from tf_agents.specs import array_spec
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.utils import common


# Load the CSV file into a pandas DataFrame
#file_path = r'C:\\Users\\s976791601\\Downloads\\predictors.csv'
file_path = r'/content/prepared_2.csv'
data = pd.read_csv(file_path)

target_column = 'target'
y = data[target_column]

data = data.drop(target_column, axis=1)

#int64_cols = list(data.select_dtypes(include='int64'))
#data[int64_cols] = data[int64_cols].astype('float64')


In [ ]:
data.dtypes

ATUAL_soma                               int64
P_2n_7_str                              object
dz3_dz4_dist_fileiras_int                int64
dz2_dz5_dist_fileiras_int                int64
maior_qtde_sorteados_fileira_int         int64
fileira_maior_qtde_sorteado_int          int64
maior_qtde_sorteados_ocorrencias_int     int64
dist_entre_fileiras_com_zero_int         int64
qtde_fileiras_sorteadas                  int64
balanceamento_p_2n_7_str                object
soma_mediana_maior_geral_str            object
faixa_soma_5_layers_str                 object
dtype: object

In [ ]:
# Separate predictors into categorical and numerical
categorical_predictors = data.select_dtypes(include=['object']).columns.tolist()
numerical_predictors = data.select_dtypes(include=['number']).columns.tolist()

In [ ]:
# Display the lists of categorical and numerical predictors
print("Categorical Predictors:", categorical_predictors)
print("Numerical Predictors:", numerical_predictors)

Categorical Predictors: ['P_2n_7_str', 'balanceamento_p_2n_7_str', 'soma_mediana_maior_geral_str', 'faixa_soma_5_layers_str']
Numerical Predictors: ['ATUAL_soma', 'dz3_dz4_dist_fileiras_int', 'dz2_dz5_dist_fileiras_int', 'maior_qtde_sorteados_fileira_int', 'fileira_maior_qtde_sorteado_int', 'maior_qtde_sorteados_ocorrencias_int', 'dist_entre_fileiras_com_zero_int', 'qtde_fileiras_sorteadas']


In [ ]:
# Preprocessing for numerical data
numerical_transformer = StandardScaler()

# Preprocessing for categorical data
categorical_transformer = OneHotEncoder(drop='first')

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_predictors),
        ('cat', categorical_transformer, categorical_predictors)
    ])

X = data
X_preprocessed = preprocessor.fit_transform(X)

In [ ]:
class PredictorSuggestionEnvironment(py_environment.PyEnvironment):
    def __init__(self, X, y):
        self._X = X
        self._y = y
        self._current_step = 0
        self._episode_ended = False
        self._state = self._convert_to_sparse_tensor(self._X[self._current_step])

        # Define action and observation specs
        self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.float64, minimum=0, maximum=self._X.shape[0] - 1, name='action')
        self._observation_spec = array_spec.BoundedArraySpec(shape=self._X.shape[1:], dtype=np.float64, minimum=-np.inf, maximum=np.inf, name='observation')

    def _convert_to_sparse_tensor(self, matrix):
        if issparse(matrix):
            matrix = tf.sparse.from_dense(matrix.shape[0])
        return matrix

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self._current_step = 0
        self._episode_ended = False
        self._state = self._convert_to_sparse_tensor(self._X[self._current_step])
        return ts.restart(self._state)

    def _step(self, action):
        if self._episode_ended:
            # The last action ended the episode. Ignore the current action.
            return self.reset()

        next_step = self._current_step + 1
        next_state = self._X[next_step].todense() if issparse(self._X) else self._X[next_step]
        reward = float(self._y[next_step])

        # Update state and check if the episode is done
        self._current_step = next_step
        self._episode_ended = next_step == self._X.shape[0] - 1

        if self._episode_ended:
            return ts.termination(next_state, reward)
        else:
            return ts.transition(next_state, reward=reward, discount=1.0)



In [ ]:
# Assuming X_preprocessed and y are your predictors and target variables
# Assuming you have already created the environment class

X_numpy = X_preprocessed
y_numpy = y.to_numpy()

In [ ]:
environment = PredictorSuggestionEnvironment(X_preprocessed, y)

# Convert the environment to TensorFlow environment
train_environment = tf_py_environment.TFPyEnvironment(environment)

# Define the Q-network
fc_layer_params = (100,)
q_net = q_network.QNetwork(
    train_environment.observation_spec(),
    train_environment.action_spec(),
    fc_layer_params=fc_layer_params,
)

# Define the DQN agent
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-3)
train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_environment.time_step_spec(),
    train_environment.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter,
)
agent.initialize()

# Define the training parameters
num_iterations = 1000
collect_steps_per_iteration = 1
replay_buffer_capacity = 1000
batch_size = 1000

# Create a replay buffer
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_environment.batch_size,
    max_length=replay_buffer_capacity,
)

# Create the data collector
collect_driver = dynamic_episode_driver.DynamicEpisodeDriver(
    train_environment, agent.collect_policy, observers=[replay_buffer.add_batch], num_episodes=collect_steps_per_iteration
)

# Training loop
for _ in range(num_iterations):
    # Collect data
    collect_driver.run()

    # Sample a batch of data from the replay buffer
    dataset = replay_buffer.as_dataset(num_parallel_calls=3, sample_batch_size=batch_size, num_steps=2).prefetch(3)

    # Train the agent
    experience, unused_info = next(iter(dataset))
    train_loss = agent.train(experience)


KeyboardInterrupt: ignored

In [ ]:
def suggest_predictors(state):
    action_step = agent.policy.action(tf.convert_to_tensor(state))
    action = action_step.action.numpy()[0]
    suggested_predictors = predictor_space[action]
    return suggested_predictors


